In [ ]:
import time
!pip install beautifulsoup4
!pip install requests
!pip install spacy
!pip install trafilatura


In [9]:
from bs4 import BeautifulSoup
import json
import os
import numpy as np
import requests
import re
import urllib
from requests.models import MissingSchema
import spacy
import trafilatura
import time
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)  #%%




In [3]:
urls = ['https://website.understandingdata.com/',
        'https://sempioneer.com/', ]
data = {}

for url2 in urls:
    # 1. Obtain the response:
    resp = requests.get(url2)

    # 2. If the response content is 200 - Status Ok, Save The HTML Content:
    if resp.status_code == 200:
        data[url2] = resp.text


In [11]:
def beautifulsoup_extract_text_fallback(response_content):
    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a
    single URL.
    '''

    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')

    # Finding the text:
    text = soup.find_all(text=True)

    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head',
        'input',
        'script',
        'style', ]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)

    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()


def parse_page(link):
    print(f'parse_page: {link}')
    downloaded_url = trafilatura.fetch_url(link, no_ssl=True)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=False, include_comments=False,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=False,
                                date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        process_page(downloaded_url)
        return downloaded_url, json_output['text']
    else:
        try:
            r = requests.get(link)
            # We will only extract the text from successful requests:
            if r.status_code == 200:

                process_page(r)

                return r.content.decode('utf-8'), beautifulsoup_extract_text_fallback(r.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return "", np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return "", np.nan


def process_page(response):
    all_relative = re.findall('href=(\'|")(.*?)(\'|")', response.content.decode('utf-8'))
    for aa in all_relative:
        if 'http' in aa[1]:
            continue
        push_url('http://fsb.ru/' + aa[1])
    all_abs = re.findall('href=(\'|")(http.*?)(\'|")', response.content.decode('utf-8'))
    for aa in all_relative:
        if not 'http' in aa[1]:
            continue
        push_url(aa[1])


urls = []
done = []
done_urls = []


def push_url(url):
    global urls, done_urls
    if url not in done_urls:
        print(f'get {url} ... [{len(done)}]', end='')

        try:
            page, text = parse_page(urllib.parse.quote(url))
        except Exception as e:
            print(f'exception: {e}')
        else:
            print(f'{len(page)}')
            time.sleep(1)
            done_urls.append(url)
            done.append({'url': url, 'page': page, 'text': text})


single_url = 'http://www.fsb.ru/'
page, text = parse_page(single_url)
print(urls)


parse_page: http://www.fsb.ru/


AttributeError: 'str' object has no attribute 'content'